<a href="https://colab.research.google.com/github/PondKann/CXR-Project/blob/main/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### import

In [103]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [104]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from imutils import paths
import argparse

#### Load data

In [105]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [106]:
train_dir = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/train'
validation_dir = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/test'

In [107]:
train_normal_dir = os.path.join(train_dir,'NORMAL')
train_pneumonia_dir = os.path.join(train_dir,'PNEUMONIA')
validation_normal_dir = os.path.join(validation_dir,'NORMAL')
validation_pneumonia_dir = os.path.join(validation_dir,'PNEUMONIA')

#### สร้างตารางเก็บชื่อภาพ และโรค 2
ข้อมูลในไฟล์ train ทั้ง normal และ pneunomia มารวมอยู่ในไฟล์ all_train แล้วลองนำมาทำ data augment

In [139]:
train_normal_dir = os.path.join(train_dir,'NORMAL')
train_pneumonia_dir = os.path.join(train_dir,'PNEUMONIA')

In [140]:
all_train = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/all_train'

สร้าง data dict ของรูป

In [202]:
normal = []
for i in range(len(os.listdir(train_normal_dir))):
    img = os.listdir(train_normal_dir)[i]
    normal.append(img)
tnormal = pd.DataFrame({'ImageName':[normal][0],
                                       'Type1': "Normal" ,
                                       'Type2': 0                      })

In [205]:
pneumonia = []
for i in range(len(os.listdir(train_pneumonia_dir))):
    img = os.listdir(train_pneumonia_dir)[i]
    pneumonia.append(img)
tpneumonia = pd.DataFrame({'ImageName':[pneumonia][0],
                                       'Type1': "Pneumonia" ,
                                       'Type2': 1                        })

In [206]:
data_train = pd.concat([tnormal, tpneumonia])
data_train.head(3)

,ImageName,Type1,Type2
0,IM-0006-0001.jpeg,Normal,1
1,IM-0003-0001.jpeg,Normal,1
2,IM-0001-0001.jpeg,Normal,1


### แบบที่ 1 

####แบ่งข้อมูล

In [207]:
train_img = []
for ImageName in tqdm(data_train['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test2
    image_path = all_train +'/' + ImageName                       # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))
    train_img.append(img)


100%|██████████| 148/148 [00:08<00:00, 17.99it/s]


In [208]:
data = np.array(train_img, dtype="float32") / 255.0

In [210]:
labels = np.array(data_train[['Type2']], dtype="float32")           #shape (148, 1) ค่าเป็น 0. กับ 1.  รัน model2.fit  ได้ ------->  แต่จะรัน classification_report หลังจาก predict แล้วมันไม่ถูก

In [ ]:
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, num_classes=2)                         #shape (148, 2) ค่าเดิม 0. กับ 1.  หลังจากรันจะเป็น [0., 1.] กับ [1., 0.]  ------->  แต่จะรัน model2.fit ไม่ได้

In [ ]:
labels.shape

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, random_state=42)

In [ ]:
testY

ไม่ต้องรันอันนี้

In [ ]:
data = np.array(train_img, dtype="float32") / 255.0
labels = np.array(data_train['Type'], dtype="float32")

#le = LabelEncoder()
#labels = le.fit_transform(labels)
#labels = to_categorical(labels,2)


(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, random_state=42)

In [ ]:
testY.argmax(axis=1)

#### Data Augmentation

In [183]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [184]:
# initialize an our data augmenter as an "empty" image data generator
aug = ImageDataGenerator()

In [185]:
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.15,
		height_shift_range=0.15,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

DenseNet201

In [186]:
base_model2 = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [187]:
base_model2.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model2.output)
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(average_pooling_layer)
model2 = tf.keras.models.Model(inputs=base_model2.input, outputs=prediction_layer)

In [188]:
model2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [189]:
# train the network

H = model2.fit(
	x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=2)

Epoch 1/2
13/13 [==============================] - 61s 3s/step - loss: 0.8035 - accuracy: 0.4563 - val_loss: 0.7682 - val_accuracy: 0.5676
Epoch 2/2
13/13 [==============================] - 46s 4s/step - loss: 0.7684 - accuracy: 0.4951 - val_loss: 0.7385 - val_accuracy: 0.5135


In [190]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model2.predict(x=testX.astype("float64"), batch_size=BS)

[INFO] evaluating network...


In [191]:
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        37

    accuracy                           1.00        37
   macro avg       1.00      1.00      1.00        37
weighted avg       1.00      1.00      1.00        37



In [ ]:
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1)))

In [201]:
predictions

array([[0.38584822],
       [0.4530935 ],
       [0.46724758],
       [0.35764468],
       [0.47683406],
       [0.45321348],
       [0.4392964 ],
       [0.4043297 ],
       [0.55491   ],
       [0.39272323],
       [0.519612  ],
       [0.36700606],
       [0.38595742],
       [0.46534073],
       [0.35634953],
       [0.31031972],
       [0.3277316 ],
       [0.48269597],
       [0.66824466],
       [0.33238962],
       [0.40470126],
       [0.2787732 ],
       [0.5479393 ],
       [0.33730817],
       [0.3645737 ],
       [0.4646588 ],
       [0.4624538 ],
       [0.30811423],
       [0.306582  ],
       [0.45070007],
       [0.49873516],
       [0.3456831 ],
       [0.43098775],
       [0.5204921 ],
       [0.47481006],
       [0.22370684],
       [0.35386047]], dtype=float32)

In [200]:
testY

array([[1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.]], dtype=float32)

In [195]:
testY.argmax(axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [196]:
predictions.argmax(axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

#### Fine tune

In [ ]:
len(base_model2.layers)

In [ ]:
base_model2.trainable = True
for layer in base_model2.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

In [ ]:
model2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
#train
model2.fit_generator(train_generator, epochs=10, 
                    validation_data=valid_generator,
                    callbacks=[callback2])

### แบบที่ 2

####แบ่งข้อมูล

In [214]:
train_img = []
for ImageName in tqdm(data_train['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test2
    image_path = all_train +'/' + ImageName                       # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))
    train_img.append(img)


100%|██████████| 148/148 [00:07<00:00, 20.77it/s]


In [215]:
data = np.array(train_img, dtype="float32") / 255.0

In [235]:
labels = np.array(data_train[['Type1']])           #shape (148, 1) ค่าเป็น 0. กับ 1.  รัน model2.fit  ได้ ------->  แต่จะรัน classification_report หลังจาก predict แล้วมันไม่ถูก

In [237]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
#อิมพอร์ต LabelBinarizer ของ sklearn มาใช้
labels = lb.fit_transform(labels)
#เรียกใช้ฟังก์ชั่น fit_transform เพื่อแปลง labels เป็นเลขฐานสอง

In [239]:
lb.classes_

array(['Normal', 'Pneumonia'], dtype='<U9')

In [240]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, random_state=42)

In [241]:
base_model2 = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [242]:
base_model2.trainable=False

In [243]:
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model2.output)
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(average_pooling_layer)

In [244]:
model2 = tf.keras.models.Model(inputs=base_model2.input, outputs=prediction_layer)

In [245]:
model2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [246]:
callback2 = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=2,
    mode="max")

In [ ]:
(trainX, testX, trainY, testY) 

In [247]:
model2.fit(trainX, trainY, epochs=5 )

Epoch 1/5
4/4 [==============================] - 40s 6s/step - loss: 0.9484 - accuracy: 0.4865
Epoch 2/5
4/4 [==============================] - 25s 6s/step - loss: 0.8860 - accuracy: 0.4505
Epoch 3/5
4/4 [==============================] - 26s 6s/step - loss: 0.8440 - accuracy: 0.4324
Epoch 4/5
4/4 [==============================] - 26s 6s/step - loss: 0.8132 - accuracy: 0.4324
Epoch 5/5
4/4 [==============================] - 26s 6s/step - loss: 0.7870 - accuracy: 0.4144


In [249]:
predY = model2.predict(testX)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
#อิมพอร์ต accuracy_score กับ classification_report จาก sklearn.metrics
accuracy_score(testY, predY)
#จากนั้นมาเรียกใช้ฟังก์ชั่น accuracy_score เพื่อดูความแม่นของข้อมูลที่ predict กัน

In [ ]:
print(classification_report(testY.argmax(axis=1) , predY.argmax(axis=1) , target_names=lb.classes_))
#เราลองมาดูผลการ classification อย่างละเอียดกัน

In [229]:
model2.evaluate(trainX, trainY)

4/4 [==============================] - 30s 6s/step - loss: 0.1073 - accuracy: 1.0000


[0.10726463794708252, 1.0]

In [232]:
predY = model2.predict(testX)

In [231]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
accuracy_score()

In [233]:
accuracy_score(testY, predY)

ValueError: ignored

In [234]:
print(classification_report(testY, predY, target_names=lb.classes_))

NameError: ignored